# 3.3 线性回归的简洁实现

在本节中，我们将介绍如何通过使用深度学习框架 `MindSpore` 来简洁地实现3.2节中的线性回归模型。

### 3.3.1 ~ 3.3.2 生成和读取数据集

mindspore.dataset提供了部分常用数据集和标准格式数据集的加载接口。对于MindSpore暂不支持直接加载的数据集，可以通过构造自定义数据集类或自定义数据集生成函数的方式来生成数据集，然后通过mindspore.dataset.GeneratorDataset接口实现自定义方式的数据集加载。

在用户自定义数据集类中须要自定义的类函数如下：

`__init__`：定义数据初始化等操作，在实例化数据集对象时被调用。

`__getitem__`：定义该函数后可使其支持随机访问，能够根据给定的索引值index，获取数据集中的数据并返回。数据返回值类型是由`NumPy数组组成的Tuple`。

`__len__`：返回数据集的样本数量。

在完成自定义数据集类之后，可以通过GeneratorDataset接口按照用户定义的方式加载并访问数据集样本。

In [1]:
import mindspore as ms
import mindspore.ops as ops
import mindspore.dataset as ds
from mindspore import Tensor

class SyntheticData():
    def __init__(self, true_w, true_b):
        self.data, self.label = self.synthetic_data(true_w, true_b, 1000)

    def __getitem__(self, index):
        return self.data[index], self.label[index]
    
    def __len__(self):
        return len(self.label)

    def synthetic_data(self, w, b, num_examples):
        """生成y=Xw+b+噪声"""
        X = ops.normal((num_examples, len(w)), Tensor(0, ms.int32), Tensor(1, ms.int32))
        y = ops.matmul(X, w) + b
        y += ops.normal(y.shape, Tensor(0, ms.int32), Tensor(0.01, ms.float32))
        return X.asnumpy(), y.reshape((-1, 1)).asnumpy()

def load_array(dataset_generator, column_names, batch_size, is_train=True):  
    """构造一个MindSpore数据迭代器。"""
    dataset = ds.GeneratorDataset(dataset_generator, column_names, shuffle=is_train)
    dataset = dataset.batch(batch_size)
    return dataset

true_w = Tensor([2, -3.4], dtype=ms.float32)
true_b = Tensor(4.2, dtype=ms.float32)
batch_size = 10
dataset_generator = SyntheticData(true_w, true_b)
dataset = load_array(dataset_generator, ["data", "label"], batch_size)

next(iter(dataset))


[Tensor(shape=[10, 2], dtype=Float32, value=
 [[2.03820133e+000, 3.46018106e-001],
  [-2.46061158e+000, 8.58924806e-001],
  [2.68145293e-001, -5.11002839e-001],
  ...
  [2.55166006e+000, -2.10053772e-001],
  [-3.04733604e-001, -1.05280924e+000],
  [-7.93688744e-003, -4.06326912e-002]]),
 Tensor(shape=[10, 1], dtype=Float32, value=
 [[7.09302092e+000],
  [-3.63361025e+000],
  [6.46829653e+000],
  ...
  [1.00384464e+001],
  [7.16260481e+000],
  [4.32362318e+000]])]

### 3.3.3 ~ 3.3.4  定义模型并初始化模型参数

In [2]:
import mindspore.nn as nn
from mindspore.common.initializer import Normal

net = nn.SequentialCell([nn.Dense(2, 1, weight_init=Normal(0.01, 0), bias_init='zero')])

### 3.3.5 定义损失函数

计算均方误差使用的是 `MSELoss` 类，也称为平方 $L_2$ 范数。 默认情况下，它返回所有样本损失的平均值。

In [3]:
loss = nn.MSELoss()

将 net 与 loss 连接。

In [4]:
net_with_loss = nn.WithLossCell(net, loss)

### 3.3.6 定义优化算法

In [5]:
optimer = nn.SGD(net.trainable_params(), learning_rate=0.03)

### 3.3.7 训练

In [6]:
trainer = nn.TrainOneStepCell(net_with_loss, optimer)
num_epochs = 3
dataset_iter = dataset.create_tuple_iterator(num_epochs=num_epochs)
for epoch in range(num_epochs):
    for data in dataset_iter:
        l = trainer(*data)
    l = net_with_loss(ms.Tensor(dataset_generator.data), ms.Tensor(dataset_generator.label))
    print(f'epoch {epoch + 1}, loss {l.asnumpy():f}')

epoch 1, loss 0.000224
epoch 2, loss 0.000095
epoch 3, loss 0.000095


比较生成数据集的真实参数和通过有限数据训练获得的模型参数。

In [8]:
w = net[0].weight.data
print('w的估计误差: ', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差: ', true_b - b)

w的估计误差:  [-0.00043845 -0.00021815]
b的估计误差:  [-0.00012636]
